# CNN迁移学习

## 环境导入

In [1]:
## 导入 Inceptionv3 模型
from keras.applications.inception_v3 import InceptionV3, preprocess_input

## 导入建立神经网络的基本模块
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

from keras.optimizers import *
from keras.losses import categorical_crossentropy

## 导入数据增强模块
import cv2
from keras_preprocessing.image import ImageDataGenerator

# 可视化
# from keras.utils import plot_model
# from keras_visualizer import visualizer
# from IPython.display import Image, SVG, display


2023-01-06 19:50:26.709256: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## 数据准备

参数区

In [2]:
DATASET_PATH_ROOT = '/home/ubuntu/notebook/DataSets/TWITTER_IMG_SENT_2015/dataset/'
BATCH_SIZE = 512
TOTAL_EPOCH = 200

In [3]:

# 数据增强
train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rescale=1. / 255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
)

val_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rescale=1. / 255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
)

# 数据输入 #Inception V3规定大小
train_generator = train_datagen.flow_from_directory(directory=f'{DATASET_PATH_ROOT}train', target_size=(299, 299), batch_size=BATCH_SIZE)
val_generator = val_datagen.flow_from_directory(directory=f'{DATASET_PATH_ROOT}validation', target_size=(299, 299), batch_size=BATCH_SIZE)

Found 617 images belonging to 2 classes.
Found 176 images belonging to 2 classes.


## 模型设定

In [4]:
# 构建基础模型
base_model = InceptionV3(weights='imagenet', include_top=False)  #去掉最后一层

# 增加新的输出层
x = base_model.output
x = GlobalAveragePooling2D()(x)  # 添加全局平均池化层 将 MxNxC 的张量转换成 1xC 张量，C是通道数
x = Dense(1024, activation='relu')(x)  # 添加一个全连接层
predictions = Dense(2, activation='softmax')(x)  # 自定义自己的分类器，这是一个2分类的分类器
model = Model(inputs=base_model.input, outputs=predictions)  # 构建我们需要训练的完整模型

2023-01-06 19:50:28.622679: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 19:50:28.631169: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 19:50:28.632741: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 19:50:28.634462: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFl

## 迁移学习

In [ ]:
# 锁层
for layer in base_model.layers:
    layer.trainable = False

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # rmsprop

# 拟合模型
history_tl = model.fit_generator(generator=train_generator,
                                 steps_per_epoch=(617 // BATCH_SIZE),  #800
                                 epochs=TOTAL_EPOCH,  #2
                                 validation_data=val_generator,
                                 validation_steps=(176 // BATCH_SIZE),  #12
                                 class_weight=None  #不要这个
                                 )
# 保存
model.save('/data/Models/Twitter2015_iv3_tl.h5')

/tmp/ipykernel_23320/2386030165.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_tl = model.fit_generator(generator=train_generator,


Epoch 1/200


2023-01-06 19:50:58.012466: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2023-01-06 19:50:58.819916: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 32s 32s/step - loss: 0.6436 - accuracy: 0.6641
Epoch 2/200
1/1 [==============================] - 11s 11s/step - loss: 2.7404 - accuracy: 0.3281
Epoch 3/200
1/1 [==============================] - 11s 11s/step - loss: 0.7241 - accuracy: 0.4199
Epoch 4/200
1/1 [==============================] - 11s 11s/step - loss: 1.1953 - accuracy: 0.6758
Epoch 5/200
1/1 [==============================] - 11s 11s/step - loss: 1.4776 - accuracy: 0.6816
Epoch 6/200
1/1 [==============================] - 11s 11s/step - loss: 1.4792 - accuracy: 0.6602
Epoch 7/200
1/1 [==============================] - 11s 11s/step - loss: 1.1269 - accuracy: 0.6738
Epoch 8/200
1/1 [==============================] - 11s 11s/step - loss: 0.8013 - accuracy: 0.6680
Epoch 9/200
